In [ ]:
import phd
from phd.experiments import lengthen

import os
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
import seaborn as sns
import numpy as np
import nengo
import brian.hears as bh
from scipy import stats
from nengo.utils.matplotlib import rasterplot
from nengo.utils.stdlib import Timer
from IPython.display import Audio

# Some plotting niceties
plt.rc('figure', figsize=(10, 6))
sns.set_style('white')
sns.set_style('ticks')

def img(array, zscore=False):
    if zscore:
        array = stats.zscore(array, axis=0)
    plt.pcolormesh(array.T)
    plt.ylim(top=array.shape[1])
    plt.xlim(right=array.shape[0])
    plt.yticks(())
    sns.despine(left=True)
    plt.tight_layout()

def img_compare(array1, array2, zscore=(False, False)):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    img(array1, zscore[0])
    plt.subplot(1, 2, 2)
    img(array2, zscore[1])
    plt.tight_layout()

timit_path = "~/phd_data/timit"
timit = phd.timit.TIMIT(timit_path)
try:
    timit.untar(os.path.expanduser("~/Dropbox/LDC93S1.tgz"))
except Exception as e:
    pass

def plot_filter(model):
    for sound in [noise, ramp, speech]:
        model.audio = sound
        net = model.build(nengo.Network(seed=0))
        with net:
            ihc_p = nengo.Probe(net.periphery.ihc, synapse=None)
            an_in_p = nengo.Probe(net.periphery.an.input, synapse=None)
        sim = nengo.Simulator(net, dt=0.0001)
        sim.run(model.t_audio)

        plt.figure(figsize=(6, 6))
        grid = gs.GridSpec(2, 2, width_ratios=[40, 1])

        ax, cax = plt.subplot(grid[0, 0]), plt.subplot(grid[0, 1])
        phd.plots.cochleogram(sim.data[ihc_p], sim.trange(), model.freqs, ax=ax, cax=cax)
        ax.set_xticks(())
        ax.set_xlabel("")
        print("IHC mean: %f" % np.mean(sim.data[ihc_p]))

        ax, cax = plt.subplot(grid[1, 0]), plt.subplot(grid[1, 1])
        phd.plots.cochleogram(sim.data[an_in_p], sim.trange(), model.freqs, ax=ax, cax=cax)
        print("AN in mean: %f" % np.mean(sim.data[an_in_p]))
        plt.tight_layout()

# --- Sounds
# Whitenoise
noise = phd.processes.WhiteNoise().run(t=1.0, d=1, dt=0.00002)
# Tone ramp
ramp = phd.processes.ToneRamp(t_ramp=1.).run(t=1.0, d=1, dt=0.00002)
# Speech
timit = phd.timit.TIMIT(timit_path)
timit.filefilt.spkr_id = "RRE0"
samples = timit.word_samples(['deadline'])
speech = samples['deadline'][0]
fs_scale = 50000. / timit.fs
resample_len = int(speech.shape[0] * fs_scale)
speech = lengthen(speech, resample_len)

# --- Model
model = phd.sermo.AuditoryFeatures()
model.cepstra.n_neurons = 1  # Just looking at periphery right now
model.fs = 50000  # All extended to fs=50000 for better comparison

In [ ]:
%%javascript
if($(IPython.toolbar.selector.concat(' > #kill-run-first')).length == 0){
  IPython.toolbar.add_buttons_group([
    {
      'label'   : 'kill and run-first',
      'icon'    : 'fa fa-angle-double-down',
      'callback': function(){
        IPython.notebook.kernel.restart();
        $(IPython.events).one('kernel_ready.Kernel', function(){
          var idx = IPython.notebook.get_selected_index();
          IPython.notebook.select(0);
          IPython.notebook.execute_cell();
          IPython.notebook.select(idx);
        });
      }
    }
  ], 'kill-run-first');
}

## Utterances

In [ ]:
# Whitenoise
Audio(data=noise.ravel(), rate=50000)

In [ ]:
# Ramping tone
Audio(data=ramp.ravel(), rate=50000)

In [ ]:
# Speech (from TIMIT)
Audio(data=speech.ravel(), rate=50000)

## Gammatone

In [ ]:
# Impulse response
t = np.linspace(0, 0.015, 500)
dt = t[1] - t[0]
f = 1000
order = 4
b = 1.019
erb_f = 24.7 + 0.108 * f
ir = (np.power(t, order-1) 
      * np.exp(-2 * np.pi * b * erb_f * t)
      * np.cos(2 * np.pi * f * t)) * 4.7e8
plt.plot(t, ir)
sns.despine()
plt.axis('tight');

In [ ]:
model.periphery.auditory_filter = 'gammatone'
plot_filter(model)

## Log Gammachirp

In [ ]:
# Impulse response
t = np.linspace(0, 0.015, 500)
dt = t[1] - t[0]
f = 1000
b = 1.019
erb_f = 24.7 + 0.108 * f
c = -1
ir = (np.power(t, 2) 
      * np.exp(-2 * np.pi * b * erb_f * t)
      * np.cos(2 * np.pi * (f * t + c * np.log(t))))
plt.plot(t, ir)
sns.despine()
plt.axis('tight');

In [ ]:
model.periphery.auditory_filter = 'log_gammachirp'
plot_filter(model)

## Dual resonance nonlinear

In [ ]:
model.periphery.auditory_filter = 'dual_resonance'
plot_filter(model)

## Compressive Gammachirp

In [ ]:
model.periphery.auditory_filter = 'compressive_gammachirp'
plot_filter(model)

## Tan Carney

In [ ]:
model.periphery.auditory_filter = 'tan_carney'
plot_filter(model)